In [3]:
!pip install xlsxwriter

In [2]:
from __future__ import division
from PIL import Image
from os import listdir
from os.path import isfile, join
from numpy.lib.stride_tricks import as_strided
from heapq import nlargest

import SPM_DVT as sd
import itertools
import csv
import numpy as np
import data_pull as dp
import os
import shutil
#import train_mnist as tm
import output_mnist as om
import load_mnist as lm
import itertools
import data_pull as dp
import math
import operator
import numpy as np

from PIL import Image
from keras.datasets import mnist
from matplotlib import pyplot as plt
from collections import Counter


import keras
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D
from keras.utils import plot_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from matplotlib import pyplot
from numpy import expand_dims
from skimage import io

def run_SPM(filter1, filter2, dir):
    #directory
    #array augmentation
    array_1 = sd.resize_array(np.load(dir + filter1))
    array_2 = sd.resize_array(np.load(dir + filter2))

    #print(array_1.shape)

    #degree
    degree = sd.degree(array_1)
    max_1 = np.amax(array_1)
    max_2 = np.amax(array_2)
    v1 = 512/max_1
    v2 = 512/max_2
    array_1 = v1 * array_1
    array_2 = v2 * array_2

    answers_dict, diff_dict = sd.pyr_all(array_1, array_2, degree)

    final_dict = {}
    for key in answers_dict:
        if answers_dict[key] > [0]:
            final_dict[key] = answers_dict[key]

    #print(final_dict)
    return dp.k_largest(40, final_dict,'y')


ModuleNotFoundError: ignored

In [ ]:

(X_train,Y_train),(X_test,Y_test)  = mnist.load_data()

X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))
X_test  = X_test/255
X_test  = X_test.astype('float')

X_test = X_test[:1000,:,:,:]

data_dir = 'data/'
#classes = ['low/', 'high/','miss/','all/']
classes = ['high/','miss/','all/']
labels = [0,1,2,3,4,5,6,7,8,9]

In [ ]:

strategy = tf.distribute.MirroredStrategy()
(X_train,Y_train),(X_test,Y_test)  = mnist.load_data()

X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2],1))

X_train = X_train/255
X_train = X_train.astype('float')
X_train[0].shape

X_train = X_train[:4000,:,:,:]
Y_train = Y_train[:4000]

"""# **BUILD THE NETWORK**"""
np.random.seed(0)
model = Sequential()
model.add(Conv2D(16,input_shape=(28,28,1),kernel_size=(3,3),activation='relu',padding='same'))
#model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
#model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
#model.add(MaxPooling2D())

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Dense(10,activation='softmax'))

model.summary()
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

model.fit(X_train,Y_train,batch_size=10,epochs=5,validation_split=0.1,shuffle=True)

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        4640      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 98,4

##Spatial pyramid matching

In [ ]:
all_selected_labels = {}

#user input into list - 4 5 -> ['4','5']
user_input = [str(x) for x in input("Run pair-wise comparison on which label? (0-9)").split()]

#comparison run seperately for each input label
for inputs in user_input:

    dp.delete_folders(data_dir,classes)

    #sampler - sample from each class
    #other_arrays - low, high, missed
    sampler, other_arrays = om.run()


    sampled = []
    for neurons in sampler[0]:
        sampled.append(neurons[1])

    lm.save_arrays(sampled, data_dir + sampler[1])
    
    for array in other_arrays:
        dp.sort_arrays(array[0], inputs, data_dir + array[1])

    all_freq = []
    frequencies = {}

    #Biulding Dictionaries of neuron frequency
    for c in classes:
        data_dir_list = os.listdir(data_dir + c)
        all_neurons = []
        for x,y in itertools.combinations(data_dir_list, 2):
            new_neurons = run_SPM(x,y, data_dir+c)
            all_neurons += new_neurons
        print(c)
        frequency = dp.CountFrequency(all_neurons)
        all_freq.append(frequency)
        #frequency2 = {k:v for k, v in frequency.items()}

        # dictionary for the frequency of activations for most popular neurons
        frequencies[c[:-1]] = frequency
        #print(frequencies)
    all_selected_labels[inputs] = frequencies

print(all_selected_labels)

Run pair-wise comparison on which label? (0-9)2
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
high/
 49 :  190
 70 :  190
 65 :  190
 25 :  190
 66 :  190
 31 :  190
 97 :  190
 41 :  190
 74 :  188
 2 :  190
 11 :  190
 73 :  190
 120 :  190
 92 :  171
 0 :  180
 91 :  175
 113 :  93
 117 :  190
 110 :  190
 87 :  189
 94 :  186
 79 :  147
 69 :  62
 44 :  63
 104 :  155
 21 :  44
 53 :  175
 62 :  68
 43 :  177
 102 :  143
 14 :  154
 59 :  137
 125 :  183
 19 :  171
 85 :  17
 108 :  108
 103 :  11
 22 :  32
 28 :  90
 60 :  155
 30 :  81
 57 :  27
 5 :  39
 58 :  46
 100 :  50
 52 :  64
 80 :  49
 83 :  15
 33 :  28
 50 :  73
 82 :  118
 67 :  106
 101 :  73
 37 :  138
 18 :  115
 64 :  77
 55 :  95
 71 :  41
 126 :  50
 13 :  73
 63 :  11
 56 :  45
 38 :  2
 12 :  97
 114 :  85
 45 :  4
 9 :  20
 107 :  17
 96 :  7
 48 :  1
 61 :  1
 88 :  10
 3 :  6
 27 :  2
miss/
 70 :  190
 49 :  190
 65 :  190
 25 :  190
 66 :  190
 97 :  190
 41 :

##TF-IDF



In [ ]:

max_high = {}
max_miss = {}

if len(all_selected_labels) > 0:
    tf_idf_value = input("Run tf-idf? (Y/N)")
    if tf_idf_value == 'Y' or tf_idf_value == 'y':
        for inputs in all_selected_labels:
            for key in inputs:
                frequencies = all_selected_labels[key]
                #print(key)
                high = frequencies['high']
                #low = frequencies['low']
                alll = frequencies['all'] 
                miss = frequencies['miss']

                group = [high, miss, alll]


                #### Build of all similarities found during pair-wise compairison for use during tf-idf calculations ###
                main = {}
                for dictionary in all_freq:
                    main = dp.mergeDict(main, dictionary)

                act_frequency = sum(all_freq[0].values()) + sum(all_freq[1].values()) + sum(all_freq[2].values())

                tf_high = {}
                tf_miss = {}
                tf_all ={}
                for key in main:
                    count = 0
                    for g in group:
                        #if key in g and g[key] >= 145:
                        if key in g:
                            count +=1
                    #print(count)
                    #print('key: ', key)
                    for f in frequencies:

                        #if key in frequencies[f] and frequencies[f][key] >= 145:
                        if key in frequencies[f]:
                            #print(f)
                            #print(f, ' value: ' , (frequencies[f][key] / sum(frequencies[f].values()))  * np.log((3 / count) + 1))
                            if f == 'high':
                                tf_high[key] = (frequencies[f][key] / sum(frequencies[f].values()))  * np.log((3 / count) + 1)
                            if f == 'miss':
                                tf_miss[key] = (frequencies[f][key] / sum(frequencies[f].values()))  * np.log((3 / count) + 1)
                            if f == 'all':
                                tf_all[key] = (frequencies[f][key] / sum(frequencies[f].values()))  * np.log((3 / count) + 1)    
                
                #print(tf_high)
                #print(tf_miss)
                #print(tf_all)

                for key in tf_all:
                    if key in tf_miss:
                        if tf_all[key] * 1.2 > tf_miss[key]:
                            del tf_miss[key]
                    if key in tf_high:
                        if tf_all[key] * 1.2 > tf_high[key]:
                            del tf_high[key]
                
                act_frequency = sum(alll.values())

                Missing_Activations = []
                #print('inputs: ', inputs)
                max_list_high = []
                max_list_miss = []
                m_high = 0
                m_miss = 0

                #print('high')
                for key in tf_high:
                    #if key not in low.keys() and key not in miss.keys():
                    #print('high')
                    if key not in tf_miss:
                        #print(key, tf_high[key])
                        if tf_high[key] > m_high:
                            max_high[inputs] = [inputs, key, tf_high[key]]
                        max_list_high.append([inputs, key, tf_high[key]])

                    elif tf_high[key] > tf_miss[key]:
                        if tf_high[key] > m_high:
                            max_high[inputs] = [inputs, key, tf_high[key]]
                        #print(key, tf_high[key])
                        max_list_high.append([inputs, key, tf_high[key]])

                #print('miss')
                for key in tf_miss:
                    #print('miss')
                    if key not in tf_high:
                        #print(key, tf_miss[key])
                        if tf_miss[key] > m_miss:
                            max_miss[inputs] = [inputs, key, tf_miss[key]]
                        max_list_miss.append([inputs, key, tf_miss[key]])
                    elif tf_miss[key] > tf_miss[key]:
                        #print(key, tf_miss[key])
                        if tf_miss[key] > m_miss:
                            max_miss[inputs] = [inputs, key, tf_miss[key]]
                        max_list_miss.append([inputs, key, tf_miss[key]])


print('only prints max value')
print('{input class, [input class, tf_idf value]}')
print('max_high: ', max_high)
print('max_miss: ', max_miss)

Run tf-idf? (Y/N)y
only prints max value
{input class, [input class, tf_idf value]}
max_high:  {'2': ['2', 96, 0.0012768500694525307]}
max_miss:  {'2': ['2', 48, 0.0013684137547248195]}
